In [1]:
# Import the requests library.
import requests
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import numpy as np
import awswrangler as wr

In [2]:
# Create variables for each of the data files
nyt_data = 'https://group3data.s3.us-west-2.amazonaws.com/us-counties.csv'
hospital_data = 'https://group3data.s3.us-west-2.amazonaws.com/COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_Facility.csv'
vaccination_data = 'https://group3data.s3.us-west-2.amazonaws.com/COVID-19_Vaccinations_in_the_United_States_County.csv'

In [3]:
# read in data from S3 bucket and create dataframes
nyt_data_df = pd.read_csv(nyt_data)
hospital_data_df = pd.read_csv(hospital_data)
vaccination_data_df = pd.read_csv(vaccination_data)

In [4]:
# Convert the date column in the hospital data to a datetime object using pd.to_datetime
hospital_data_df['collection_week'] = pd.to_datetime(hospital_data_df['collection_week'])

In [5]:
# Sort the hospital data by date
hospital_data_df = hospital_data_df.sort_values(by='collection_week')
hospital_data_df

,hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,...,previous_day_admission_pediatric_covid_confirmed_7_day_coverage,previous_day_admission_adult_covid_suspected_7_day_coverage,previous_day_admission_pediatric_covid_suspected_7_day_coverage,previous_week_personnel_covid_vaccinated_doses_administered_7_day,total_personnel_covid_vaccinated_doses_none_7_day,total_personnel_covid_vaccinated_doses_one_7_day,total_personnel_covid_vaccinated_doses_all_7_day,previous_week_patients_covid_vaccinated_doses_one_7_day,previous_week_patients_covid_vaccinated_doses_all_7_day,is_corrected
324747,110045,2020-07-31,GA,110045,NORTHEAST GEORGIA MEDICAL CENTER BARROW,316 NORTH BROAD STREET,WINDER,30680.0,Short Term,13013.0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,False
319367,180045,2020-07-31,KY,180045,ST ELIZABETH FLORENCE,4900 HOUSTON ROAD,FLORENCE,41042.0,Short Term,21015.0,...,7,7,7,NaN,NaN,NaN,NaN,NaN,NaN,False
319371,100168,2020-07-31,FL,100168,BOCA RATON REGIONAL HOSPITAL,800 MEADOWS RD,BOCA RATON,33486.0,Short Term,12099.0,...,7,7,7,NaN,NaN,NaN,NaN,NaN,NaN,False
319374,161322,2020-07-31,IA,161322,DALLAS COUNTY HOSPITAL,610 TENTH STREET,PERRY,50220.0,Critical Access Hospitals,19049.0,...,7,7,7,NaN,NaN,NaN,NaN,NaN,NaN,False
319375,050660,2020-07-31,CA,050660,USC KENNETH NORRIS JR CANCER HOSPITAL,1441 EASTLAKE AVE,LOS ANGELES,90089.0,Short Term,6037.0,...,7,7,7,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10378,341320,2021-10-22,NC,341320,ALLEGHANY COUNTY MEMORIAL HOSPITAL,617 DOCTORS STREET,SPARTA,28675.0,Critical Access Hospitals,37005.0,...,7,7,7,NaN,NaN,NaN,NaN,NaN,NaN,False
10377,451311,2021-10-22,TX,451311,SWEENY COMMUNITY HOSPITAL,305 NORTH MCKINNEY,SWEENY,77480.0,Critical Access Hospitals,48039.0,...,7,7,7,NaN,NaN,NaN,NaN,NaN,NaN,False
6377,171318,2021-10-22,KS,171318,SEDAN CITY HOSPITAL,300 NORTH STREET,SEDAN,67361.0,Critical Access Hospitals,20019.0,...,7,7,7,0.0,12.0,-999999.0,29.0,0.0,0.0,False
10372,180011,2021-10-22,KY,180011,CHI SAINT JOSEPH LONDON,1001 SAINT JOSEPH LANE,LONDON,40741.0,Short Term,21125.0,...,7,7,7,0.0,0.0,0.0,0.0,0.0,0.0,False


In [6]:
# Convert the hospital date and fips_code column to strings
hospital_data_df.rename(columns={'collection_week': 'date'}, inplace=True)
hospital_data_df['date'] = hospital_data_df['date'].astype('str')
hospital_data_df['fips_code'] = hospital_data_df['fips_code'].astype('str')

In [7]:
# Concat the date and fips_code for new columns fips_date
hospital_data_df['fips_date'] = hospital_data_df['fips_code'] + hospital_data_df['date']

In [8]:
# Create a new df to groupby the new fips_date column so each row is unique
agg_hospital_data_df = hospital_data_df.groupby(['fips_date']).agg({'total_beds_7_day_sum':'sum','all_adult_hospital_beds_7_day_sum':'sum', 'all_adult_hospital_inpatient_beds_7_day_sum':'sum', 'inpatient_beds_used_7_day_sum':'sum', 'all_adult_hospital_inpatient_bed_occupied_7_day_sum':'sum', 
                                                                                                                       'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum':'sum', 'total_adult_patients_hospitalized_confirmed_covid_7_day_sum': 'sum', 'inpatient_beds_7_day_sum':'sum', 'total_icu_beds_7_day_sum':'sum', 
                                                                                                                      'total_staffed_adult_icu_beds_7_day_sum':'sum', 'icu_beds_used_7_day_sum':'sum', 'staffed_adult_icu_bed_occupancy_7_day_sum':'sum', 'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum':'sum', 'staffed_icu_adult_patients_confirmed_covid_7_day_sum':'sum'}).reset_index()
agg_hospital_data_df

,fips_date,total_beds_7_day_sum,all_adult_hospital_beds_7_day_sum,all_adult_hospital_inpatient_beds_7_day_sum,inpatient_beds_used_7_day_sum,all_adult_hospital_inpatient_bed_occupied_7_day_sum,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_adult_patients_hospitalized_confirmed_covid_7_day_sum,inpatient_beds_7_day_sum,total_icu_beds_7_day_sum,total_staffed_adult_icu_beds_7_day_sum,icu_beds_used_7_day_sum,staffed_adult_icu_bed_occupancy_7_day_sum,staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum,staffed_icu_adult_patients_confirmed_covid_7_day_sum
0,10001.02020-07-31,2100.0,1939.0,1688.0,1713.0,1558.0,152.0,84.0,1876.0,322.0,203.0,266.0,189.0,28.0,28.0
1,10001.02020-08-07,2100.0,1939.0,1694.0,1672.0,1548.0,112.0,71.0,1876.0,322.0,203.0,273.0,188.0,22.0,22.0
2,10001.02020-08-14,2100.0,1939.0,1210.0,1141.0,1028.0,132.0,55.0,1340.0,198.0,134.0,183.0,128.0,29.0,29.0
3,10001.02020-08-21,2100.0,1939.0,1729.0,1650.0,1502.0,93.0,57.0,1908.0,285.0,193.0,245.0,171.0,23.0,23.0
4,10001.02020-08-28,2100.0,1939.0,1939.0,1730.0,1624.0,65.0,49.0,2100.0,315.0,203.0,251.0,182.0,19.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159931,nan2021-09-24,1218.0,1204.0,1008.0,526.0,447.0,0.0,0.0,1218.0,49.0,49.0,37.0,37.0,0.0,0.0
159932,nan2021-10-01,1218.0,1204.0,1008.0,523.0,441.0,0.0,0.0,1218.0,49.0,49.0,34.0,34.0,0.0,0.0
159933,nan2021-10-08,1218.0,1204.0,1008.0,524.0,443.0,0.0,0.0,1218.0,49.0,49.0,38.0,38.0,0.0,0.0
159934,nan2021-10-15,1218.0,1204.0,1008.0,540.0,462.0,0.0,0.0,1218.0,49.0,49.0,41.0,41.0,0.0,0.0


In [14]:
# Convert the date column in the NYT data to a datetime object using pd.to_datetime
nyt_data_df['date'] = pd.to_datetime(nyt_data_df['date'])

In [15]:
# 1. Remove the data for dates before 7-31-20 and after 10-22-21
# 2. Bin the date column by weeks ending every friday
# 3. Sum the cases and deaths columns for each week binned

# Step 1:
nyt_data_filtered = nyt_data_df[nyt_data_df['date'] >= '2020-07-18']
nyt_data_filtered = nyt_data_filtered[nyt_data_filtered['date'] <= '2021-10-29']

In [16]:
# Step 2:
# Step 3:
columns_to_sum = [nyt_data_filtered['cases'], nyt_data_filtered['deaths']]
nyt_data_grouped = nyt_data_filtered.groupby(['fips', pd.Grouper(key='date', freq='W-FRI')]).agg({'cases':'sum','deaths':'sum'}).reset_index()

In [17]:
# Resort the nyt_data_grouped by date
nyt_data_grouped = nyt_data_grouped.sort_values(by='date')

In [18]:
# Set each column to a string value so we can concatonate them together 
nyt_data_grouped['fips'] = nyt_data_grouped['fips'].astype('str')
nyt_data_grouped['date'] = nyt_data_grouped['date'].astype('str')

In [19]:
# Concatonate the date column and the fips column to create fips_date
nyt_data_grouped['fips_date'] = nyt_data_grouped['fips'] + nyt_data_grouped['date']

In [69]:
x = agg_hospital_data_df['fips_date'].unique()
y = nyt_data_grouped['fips_date'].unique()

In [34]:
diff_list = list(set(y).symmetric_difference(set(x)))
diff_list

['29111.02020-11-06',
 '51007.02021-05-07',
 '51097.02021-01-29',
 '48159.02021-07-30',
 '2290.02020-10-30',
 '54071.02021-04-09',
 '48045.02021-07-02',
 '37143.02021-10-08',
 '1067.02020-12-18',
 '24011.02021-04-16',
 '21005.02020-10-23',
 '18161.02021-05-28',
 '29015.02021-07-23',
 '40053.02020-08-07',
 '26135.02021-04-16',
 '5089.02021-06-04',
 '21011.02020-09-11',
 '21075.02021-01-15',
 '26083.02021-05-21',
 '54105.02021-08-13',
 '46025.02021-04-16',
 '15005.02021-01-22',
 '51199.02020-10-23',
 '28049.02021-10-29',
 '6023.02021-10-29',
 '29087.02021-07-23',
 '17191.02021-10-29',
 '39111.02021-01-15',
 '13239.02020-12-04',
 '20011.02020-09-18',
 '47087.02020-10-23',
 '78030.02021-02-26',
 '20003.02021-10-29',
 '72011.02020-10-30',
 '72063.02021-10-29',
 '13181.02021-03-12',
 '18007.02020-12-25',
 '48311.02021-07-16',
 '55077.02021-07-30',
 '48345.02020-10-09',
 '39135.02021-06-04',
 '42067.02020-09-04',
 '51520.02021-02-26',
 '13171.02020-12-18',
 '2068.02020-09-25',
 '47135.02021-0

In [35]:
# Need to drop all the rows in the hospital data that dont occur in the nyt data...diff_list
# merged nyt data and hospital data
merged = agg_hospital_data_df.merge(nyt_data_grouped)
merged

,fips_date,total_beds_7_day_sum,all_adult_hospital_beds_7_day_sum,all_adult_hospital_inpatient_beds_7_day_sum,inpatient_beds_used_7_day_sum,all_adult_hospital_inpatient_bed_occupied_7_day_sum,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_adult_patients_hospitalized_confirmed_covid_7_day_sum,inpatient_beds_7_day_sum,total_icu_beds_7_day_sum,total_staffed_adult_icu_beds_7_day_sum,icu_beds_used_7_day_sum,staffed_adult_icu_bed_occupancy_7_day_sum,staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum,staffed_icu_adult_patients_confirmed_covid_7_day_sum,fips,date,cases,deaths
0,10001.02020-07-31,2100.0,1939.0,1688.0,1713.0,1558.0,152.0,84.0,1876.0,322.0,203.0,266.0,189.0,28.0,28.0,10001.0,2020-07-31,14925,737.0
1,10001.02020-08-07,2100.0,1939.0,1694.0,1672.0,1548.0,112.0,71.0,1876.0,322.0,203.0,273.0,188.0,22.0,22.0,10001.0,2020-08-07,15576,749.0
2,10001.02020-08-14,2100.0,1939.0,1210.0,1141.0,1028.0,132.0,55.0,1340.0,198.0,134.0,183.0,128.0,29.0,29.0,10001.0,2020-08-14,16348,756.0
3,10001.02020-08-21,2100.0,1939.0,1729.0,1650.0,1502.0,93.0,57.0,1908.0,285.0,193.0,245.0,171.0,23.0,23.0,10001.0,2020-08-21,17518,759.0
4,10001.02020-08-28,2100.0,1939.0,1939.0,1730.0,1624.0,65.0,49.0,2100.0,315.0,203.0,251.0,182.0,19.0,19.0,10001.0,2020-08-28,17907,763.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159059,9015.02021-09-24,951.0,937.0,783.0,565.0,565.0,56.0,42.0,783.0,119.0,119.0,34.0,34.0,14.0,10.0,9015.0,2021-09-24,85476,1400.0
159060,9015.02021-10-01,956.0,942.0,788.0,530.0,530.0,61.0,57.0,788.0,119.0,119.0,32.0,32.0,15.0,14.0,9015.0,2021-10-01,87031,1439.0
159061,9015.02021-10-08,956.0,942.0,788.0,557.0,557.0,87.0,85.0,788.0,119.0,119.0,32.0,32.0,20.0,20.0,9015.0,2021-10-08,88585,1474.0
159062,9015.02021-10-15,955.0,941.0,787.0,521.0,521.0,76.0,74.0,787.0,119.0,119.0,26.0,26.0,19.0,19.0,9015.0,2021-10-15,90074,1483.0


In [91]:
merged.set_index(['fips_date'])

,total_beds_7_day_sum,all_adult_hospital_beds_7_day_sum,all_adult_hospital_inpatient_beds_7_day_sum,inpatient_beds_used_7_day_sum,all_adult_hospital_inpatient_bed_occupied_7_day_sum,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_adult_patients_hospitalized_confirmed_covid_7_day_sum,inpatient_beds_7_day_sum,total_icu_beds_7_day_sum,total_staffed_adult_icu_beds_7_day_sum,icu_beds_used_7_day_sum,staffed_adult_icu_bed_occupancy_7_day_sum,staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum,staffed_icu_adult_patients_confirmed_covid_7_day_sum,fips,date,cases,deaths
fips_date,,,,,,,,,,,,,,,,,,
10001.02020-07-31,2100.0,1939.0,1688.0,1713.0,1558.0,152.0,84.0,1876.0,322.0,203.0,266.0,189.0,28.0,28.0,10001.0,2020-07-31,14925,737.0
10001.02020-08-07,2100.0,1939.0,1694.0,1672.0,1548.0,112.0,71.0,1876.0,322.0,203.0,273.0,188.0,22.0,22.0,10001.0,2020-08-07,15576,749.0
10001.02020-08-14,2100.0,1939.0,1210.0,1141.0,1028.0,132.0,55.0,1340.0,198.0,134.0,183.0,128.0,29.0,29.0,10001.0,2020-08-14,16348,756.0
10001.02020-08-21,2100.0,1939.0,1729.0,1650.0,1502.0,93.0,57.0,1908.0,285.0,193.0,245.0,171.0,23.0,23.0,10001.0,2020-08-21,17518,759.0
10001.02020-08-28,2100.0,1939.0,1939.0,1730.0,1624.0,65.0,49.0,2100.0,315.0,203.0,251.0,182.0,19.0,19.0,10001.0,2020-08-28,17907,763.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9015.02021-09-24,951.0,937.0,783.0,565.0,565.0,56.0,42.0,783.0,119.0,119.0,34.0,34.0,14.0,10.0,9015.0,2021-09-24,85476,1400.0
9015.02021-10-01,956.0,942.0,788.0,530.0,530.0,61.0,57.0,788.0,119.0,119.0,32.0,32.0,15.0,14.0,9015.0,2021-10-01,87031,1439.0
9015.02021-10-08,956.0,942.0,788.0,557.0,557.0,87.0,85.0,788.0,119.0,119.0,32.0,32.0,20.0,20.0,9015.0,2021-10-08,88585,1474.0


In [44]:
merged.dtypes

fips_date                                                                     object
total_beds_7_day_sum                                                         float64
all_adult_hospital_beds_7_day_sum                                            float64
all_adult_hospital_inpatient_beds_7_day_sum                                  float64
inpatient_beds_used_7_day_sum                                                float64
all_adult_hospital_inpatient_bed_occupied_7_day_sum                          float64
total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum    float64
total_adult_patients_hospitalized_confirmed_covid_7_day_sum                  float64
inpatient_beds_7_day_sum                                                     float64
total_icu_beds_7_day_sum                                                     float64
total_staffed_adult_icu_beds_7_day_sum                                       float64
icu_beds_used_7_day_sum                                          

In [94]:
if x[0] == merged.fips_date[0]:
    print('yes')
else:
    print('no')

yes


In [97]:
if x[0] in merged.fips_date:
    print('yes')
else:
    print('no')

no


'29111.02020-11-06'